In [8]:
from sympy import *

In [27]:
m = Matrix([
    [2,3,0],
    [0, 1, 7],
    [1, 0 , -1]
])

In [28]:
m.rref()

(Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]),
 (0, 1, 2))

In [15]:
m.nullspace()

[Matrix([
 [ 4],
 [-2],
 [ 1],
 [ 0],
 [ 0]]),
 Matrix([
 [ 1],
 [ 1],
 [ 0],
 [-1],
 [ 1]])]

In [18]:
m*m.T

Matrix([
[16,  28,  -8,  40],
[28,  78, -33,  89],
[-8, -33,  18, -34],
[40,  89, -34, 114]])

In [25]:
A = Matrix([[2,-1,1,3],[3,1,-2,-2],[1,-1,1,7],[1,5,7,13],[1,-7,-5,12]])

In [26]:
A.rref()

(Matrix([
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 0]]),
 (0, 1, 2, 3))